In [3]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
preprocessed_image = prepare_image('output/popcorn/2.jpg')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

[[('n02124075', 'Egyptian_cat', 0.5395711),
  ('n02123597', 'Siamese_cat', 0.24800476),
  ('n02127052', 'lynx', 0.1419059),
  ('n02110185', 'Siberian_husky', 0.0059532803),
  ('n04589890', 'window_screen', 0.004288093)]]

In [4]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(5,activation='softmax')(x) #final layer with softmax activation

/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [5]:
model=Model(inputs=base_model.input,outputs=preds)

In [33]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_2
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [7]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [8]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('output',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 148 images belonging to 5 classes.


In [9]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

In [14]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=20)

Epoch 1/20
4/4 [==============================] - 17s 4s/step - loss: 0.2214 - acc: 0.9609
Epoch 2/20
4/4 [==============================] - 15s 4s/step - loss: 0.1001 - acc: 0.9681
Epoch 3/20
4/4 [==============================] - 14s 4s/step - loss: 0.1118 - acc: 0.9841
Epoch 4/20
4/4 [==============================] - 14s 4s/step - loss: 0.1937 - acc: 0.9644
Epoch 5/20
4/4 [==============================] - 14s 4s/step - loss: 0.0042 - acc: 1.0000
Epoch 6/20
4/4 [==============================] - 15s 4s/step - loss: 0.1497 - acc: 0.9688
Epoch 7/20
4/4 [==============================] - 14s 4s/step - loss: 0.0105 - acc: 1.0000
Epoch 8/20
4/4 [==============================] - 14s 3s/step - loss: 0.1282 - acc: 0.9766
Epoch 9/20
4/4 [==============================] - 14s 3s/step - loss: 0.0118 - acc: 1.0000
Epoch 10/20
4/4 [==============================] - 14s 3s/step - loss: 0.0089 - acc: 1.0000
Epoch 11/20
4/4 [==============================] - 15s 4s/step - loss: 0.0506 - acc: 0.99

In [13]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [14]:
def decode_predictions(pred):
    predicted_class_indices = np.argmax(pred,axis=1)
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [(labels[k], pred[0][k]) for k in predicted_class_indices] 
    return predictions

In [17]:
import glob

#img_path = 'output/jam/10.jpg'
paths = glob.glob("output/**/*.jpg")

for img_path in paths:
    new_image = load_image(img_path)
    pred = model.predict(new_image)
    predictions = decode_predictions(pred)
    print(img_path, predictions)

output/raspberry/8.jpg [('raspberry', 1.0)]
output/raspberry/9.jpg [('raspberry', 0.9999962)]
output/raspberry/14.jpg [('muffin', 0.9582715)]
output/raspberry/15.jpg [('muffin', 1.0)]
output/raspberry/17.jpg [('raspberry', 0.99997723)]
output/raspberry/16.jpg [('muffin', 0.99396014)]
output/raspberry/12.jpg [('muffin', 1.0)]
output/raspberry/13.jpg [('muffin', 0.9999993)]
output/raspberry/11.jpg [('muffin', 0.86153007)]
output/raspberry/10.jpg [('raspberry', 0.999948)]
output/raspberry/21.jpg [('raspberry', 0.93273777)]
output/raspberry/20.jpg [('raspberry', 0.9988482)]
output/raspberry/22.jpg [('muffin', 0.93410486)]
output/raspberry/23.jpg [('muffin', 1.0)]
output/raspberry/26.jpg [('raspberry', 0.99999404)]
output/raspberry/18.jpg [('muffin', 1.0)]
output/raspberry/24.jpg [('muffin', 0.990168)]
output/raspberry/25.jpg [('muffin', 0.9998136)]
output/raspberry/19.jpg [('muffin', 1.0)]
output/raspberry/4.jpg [('raspberry', 0.84297764)]
output/raspberry/5.jpg [('raspberry', 0.9498381)]


In [35]:
img_path = "test/popcorn_test.png"
new_image = load_image(img_path)
pred = model.predict(new_image)
predictions = decode_predictions(pred)
print(img_path, predictions)

test/popcorn_test.png [('muffin', 1.0)]


In [32]:
model.save('./model/keras_model.h5')

In [8]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [21]:
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 714 variables.
INFO:tensorflow:Converted 714 variables to const ops.


In [ ]:
tf.train.write_graph(frozen_graph, "model", "cats_model.pb", as_text=False)

In [22]:
model.outputs

[<tf.Tensor 'dense_4/Softmax:0' shape=(?, 5) dtype=float32>]

In [29]:
model.inputs

[<tf.Tensor 'input_2:0' shape=(?, ?, ?, 3) dtype=float32>]

In [30]:
train_generator.class_indices

{'berry': 0, 'jam': 1, 'muffin': 2, 'popcorn': 3, 'raspberry': 4}

In [6]:
import os
MODEL_NAME = 'model'
PATH_TO_CKPT = MODEL_NAME + '/cats_model.pb'
PATH_TO_LABELS = os.path.join(MODEL_NAME, 'graph.pbtxt')
NUM_CLASSES = 5

In [9]:
classification_graph = tf.Graph()
with classification_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    classification_sess = tf.Session(graph=classification_graph)

In [10]:
from object_detection.utils import label_map_util
cats_label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
cats_categories = label_map_util.convert_label_map_to_categories(
    cats_label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
cats_category_index = label_map_util.create_category_index(cats_categories)

In [11]:
def classify_image(image_np_expanded, sess, graph):
    #image_np_expanded = np.expand_dims(frame, axis=0)
    image_tensor = graph.get_tensor_by_name('input_4:0')
    classes = graph.get_tensor_by_name('dense_12/Softmax:0')
    (classes) = sess.run(
        [classes],
        feed_dict={image_tensor: image_np_expanded})
    local_classes = np.squeeze(classes)
    min_score_thresh = .5
    print('classify image, found ', local_classes)
    for i in range(len(local_classes)):
        score = local_classes[i]
        # if score > min_score_thresh:
        class_name = cats_category_index[i+1]['name']
        print('found ', i, class_name, score)

In [24]:
img_path = "../edge-server/popcorn.jpeg"
new_image = load_image(img_path)
classify_image(new_image, classification_sess, classification_graph)

classify image, found  [7.1741763e-18 8.9007530e-27 6.5556221e-27 1.0000000e+00 2.1765903e-15]
found  0 berry 7.174176e-18
found  1 jam 8.900753e-27
found  2 muffin 6.555622e-27
found  3 popcorn 1.0
found  4 raspberry 2.1765903e-15


In [ ]:
model.load_weights('./model/keras_model.h5')

In [26]:
import tensorflowjs as tfjs

In [31]:
tf.keras.backend.cle ar_session()
tfjs_target_dir = "tfjs-models"
tfjs.converters.save_keras_model(model, tfjs_target_dir)

In [ ]:
tfjs.converters.convert_tf_saved_model